# **Group: Jaswanth Kranthi Boppana, Manav Mandal, Sayan Das, Vishnu-vardhan Reddy Kollu**

# 1. Create your own dataset for text classification. It should contain at least 1000 words in total and at least two categories with at least 100 examples per category. You can create it by scraping the web or using some of the documents you have on your computer or ChatGPT. [15 points]

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

**We have created a restaurant review dataset. In which we have postive and negative review statements given by various customers on different restaurants.**


*   **In the dataset we have 2 colums 1.Review  2. Sentiment.**
*   **The sentiment defines wheather the review is postive (1) or neagative (0)**.
* **we have 2 classes anf Each class of postive (1) and negative (0)  has 100 review instances each.**





In [3]:
csv_file_path = "restaurant_reviews_final.csv"
df = pd.read_csv(csv_file_path)

**Displaying some part of the dataset to provide a basic view how the data looks like**

In [4]:
#first 5 entries in the Dataset
df.head()

,Review,Sentiment
0,A culinary delight! My taste buds are still th...,1
1,A culinary journey that transports you to diff...,1
2,A culinary symphony of flavors! Each dish is a...,1
3,A delightful escape from the ordinary. The foo...,1
4,A five-star experience from the moment you wal...,1


In [5]:
#last 5 entries in the Dataset.
df.tail()

,Review,Sentiment
195,This place is a disgrace to the culinary world...,0
196,This place is a hidden gem! The flavors are ou...,1
197,This place is a paradise for food lovers. Each...,1
198,This place is a treasure trove of gastronomic ...,1
199,This place was a complete letdown. The food ta...,0


* **The Below code indicates that there are 2 classes (0 (neagative) and 1 (Positive )) and each has 100 instances.**   
*   **And it is evident that the reviews statements are of a good lenght, so the dataset surely has more than 1000 words in total.**



In [6]:
df['Sentiment'].value_counts()

1    100
0    100
Name: Sentiment, dtype: int64

**The info() functions gives much more clear view of the dataset.**

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Review     200 non-null    object
 1   Sentiment  200 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.2+ KB


# 2. Split the dataset into training (at least 160examples) and test (at least 40 examples) sets. [5 points]

In [9]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

**Dataset splited into train and test. The train set has 160 instances of 2 classes.**

In [10]:
train_data.shape

(160, 2)

**The test set has 40 instances of 2 classes.**

In [11]:
test_data.shape

(40, 2)

In [12]:
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [13]:
train_data=train_data.iloc[::]
test_data=test_data.iloc[::]

# 3. Fine tune a pretrained language model capable of generating text (e.g., GPT) that you can take from the Hugging Face Transformers library with the dataset your created (this tutorial could be very helpful: https://huggingface.co/docs/transformers/training). Report the test accuracy. Discuss what could be done to improve accuracy. [30 points]

In [14]:
train_data_tkn = Dataset.from_pandas(train_data)
test_data_tkn = Dataset.from_pandas(test_data)

In [15]:
train_data_tkn

Dataset({
    features: ['Review', 'Sentiment'],
    num_rows: 160
})

*   **Before we start fine tunning the model we have to tokenize the Reviews in   the dataset**.
*   **This helps the model to understand about the particular instance more easily and effectively.**
*   **We are using the "bert-base-cased model" to Tokenize the reviews text.**

In [16]:
import transformers
from transformers import AutoTokenizer
import numpy as np
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_train = tokenizer(train_data_tkn["Review"], return_tensors="np", padding=True)
tokenized_test = tokenizer(test_data_tkn["Review"], return_tensors="np", padding=True)

tokenized_train = dict(tokenized_train)
tokenized_test = dict(tokenized_test)

labels_train = np.array(train_data_tkn["Sentiment"])
labels_test = np.array(test_data_tkn["Sentiment"])

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

**We are now Fine-tunning a pretrained model on our tokenized data and labels.  Here we are using the transformers based pretrained model.**

**This is the BSC-TeMU/roberta-base-bne model trained on the squad_es v2.0.0 dataset.**  



In [56]:
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
## Model Definition
model = TFAutoModelForSequenceClassification.from_pretrained("BSC-TeMU/roberta-base-bne", from_pt=True, num_labels=2)
## Model Compilation
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.metrics.SparseCategoricalAccuracy()
model.compile(optimizer=optimizer, loss=loss, metrics=metric)

## Fitting the data
history = model.fit(tokenized_train, labels_train, epochs=20, batch_size=64)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Epoch 1/20
3/3 [==============================] - 106s 16s/step - loss: 0.9455 - sparse_categorical_accuracy: 0.4750
Epoch 2/20
3/3 [==============================] - 54s 17s/step - loss: 0.8590 - sparse_categorical_accuracy: 0.4750
Epoch 3/20
3/3 [==============================] - 53s 16s/step - loss: 0.7834 - sparse_categorical_accuracy: 0.5125
Epoch 4/20
3/3 [==============================] - 52s 16s/step - loss: 0.7301 - sparse_categorical_accuracy: 0.4875
Epoch 5/20
3/3 [==============================] - 52s 16s/step - loss: 0.6998 - sparse_categorical_accuracy: 0.5125
Epoch 6/20
3/3 [==============================] - 51s 16s/step - loss: 0.6982 - sparse_categorical_accuracy: 0.5125
Epoch 7/20
3/3 [==============================] - 52s 16s/step - loss: 0.6955 - sparse_categorical_accuracy: 0.5000
Epoch 8/20
3/3 [==============================] - 51s 16s/step - loss: 0.7087 - sparse_categorical_accuracy: 0.5125
Epoch 9/20
3/3 [==============================] - 54s 17s/step - loss: 

In [57]:
predictions= model.predict(tokenized_test)

2/2 [==============================] - 8s 952ms/step


On Fine Tunning the model on our dataset and then testing it with the test set we got the **Testing accuracy For the model as 45%**.

In [58]:
model.evaluate(tokenized_test,labels_test)

2/2 [==============================] - 13s 930ms/step - loss: 0.7022 - sparse_categorical_accuracy: 0.4500


[0.7021811604499817, 0.44999998807907104]

**We are trying to Change few parameters of the model to increase the model accracy.**

**Here we added the "nadam" optimizer and changed the learning rate to 0.01.**

In [19]:
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
## Model Definition
model_2 = TFAutoModelForSequenceClassification.from_pretrained("BSC-TeMU/roberta-base-bne", from_pt=True, num_labels=2)
## Model Compilation
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.metrics.SparseCategoricalAccuracy()
model_2.compile(optimizer='nadam', loss=loss, metrics=metric)

## Fitting the data
history = model_2.fit(tokenized_train, labels_train, epochs=20, batch_size=64)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Epoch 1/20
3/3 [==============================] - 117s 16s/step - loss: 0.8672 - sparse_categorical_accuracy: 0.4875
Epoch 2/20
3/3 [==============================] - 50s 15s/step - loss: 0.7693 - sparse_categorical_accuracy: 0.5875
Epoch 3/20
3/3 [==============================] - 52s 16s/step - loss: 0.6995 - sparse_categorical_accuracy: 0.5125
Epoch 4/20
3/3 [==============================] - 50s 15s/step - loss: 0.6998 - sparse_categorical_accuracy: 0.5000
Epoch 5/20
3/3 [==============================] - 52s 15s/step - loss: 0.7132 - sparse_categorical_accuracy: 0.5125
Epoch 6/20
3/3 [==============================] - 50s 15s/step - loss: 0.6953 - sparse_categorical_accuracy: 0.4750
Epoch 7/20
3/3 [==============================] - 51s 15s/step - loss: 0.7016 - sparse_categorical_accuracy: 0.5125
Epoch 8/20
3/3 [==============================] - 50s 15s/step - loss: 0.7095 - sparse_categorical_accuracy: 0.5125
Epoch 9/20
3/3 [==============================] - 58s 15s/step - loss: 

After changing few training parameter we can see the accuaracy of the model has been increased.

So now the **Test accuracy of the model is 55%.**

In [20]:
model_2.evaluate(tokenized_test,labels_test)

2/2 [==============================] - 16s 1s/step - loss: 0.6917 - sparse_categorical_accuracy: 0.5500


[0.6916903853416443, 0.550000011920929]

**We have also tried using one more pretrained model on the dataset. Here we have used the "bert-base-uncased" pretrained model.**

**The "bert-based-uncased" is  a BERT  transformers model pretrained on a large corpus of English data in a self-supervised fashion.**

In [59]:
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
## Model Definition
model_1 = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", from_pt=True, num_labels=3)
## Model Compilation
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.metrics.SparseCategoricalAccuracy()
model_1.compile(optimizer=optimizer, loss=loss, metrics=metric)

## Fitting the data
history = model_1.fit(tokenized_train, labels_train, epochs=20, batch_size=64)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20
3/3 [==============================] - 95s 16s/step - loss: 1.0222 - sparse_categorical_accuracy: 0.4500
Epoch 2/20
3/3 [==============================] - 53s 16s/step - loss: 0.6833 - sparse_categorical_accuracy: 0.6687
Epoch 3/20
3/3 [==============================] - 51s 15s/step - loss: 1.0633 - sparse_categorical_accuracy: 0.4313
Epoch 4/20
3/3 [==============================] - 52s 16s/step - loss: 0.6905 - sparse_categorical_accuracy: 0.5625
Epoch 5/20
3/3 [==============================] - 52s 16s/step - loss: 0.8271 - sparse_categorical_accuracy: 0.5063
Epoch 6/20
3/3 [==============================] - 53s 16s/step - loss: 0.7626 - sparse_categorical_accuracy: 0.5000
Epoch 7/20
3/3 [==============================] - 52s 16s/step - loss: 1.0041 - sparse_categorical_accuracy: 0.4437
Epoch 8/20
3/3 [==============================] - 52s 16s/step - loss: 0.7138 - sparse_categorical_accuracy: 0.5063
Epoch 9/20
3/3 [==============================] - 51s 16s/step - loss: 0

**On Training on the BERT based model we go the Accuracy as 55%**

In [60]:
model.evaluate(tokenized_test,labels_test)

2/2 [==============================] - 14s 987ms/step - loss: 0.6886 - sparse_categorical_accuracy: 0.5500


[0.6885843276977539, 0.550000011920929]

**Discussion to Improve accuracy of the pretrained model:**

The accuracy of a pretrained language model, such as BSC-TeMU/roberta-base-bne, bert-base-uncased on a text classification dataset, can be increased using a number of methods.

*   One method is to use the dataset to fine-tune the model on the particular job, modifying the hyperparameters and optimizing the loss function.

*   Another strategy is to improve the diversity of the training data by using data augmentation techniques like synonym substitution or back-translation.

*   Accuracy can also be increased by training the model with a larger corpus of data or by utilizing an ensemble of models.

*  Finally, the generalization performance of the model can also be enhanced by regularizing it using strategies like dropout or early halting.

*   We can also be specitcal about selecting a particular pretrained model. Choosing a good pretrained model and by understanding on which data it is trained helps in increasing the model accuracy on your dataset.


